# calculate rotation and transposition matrix

In [1]:
import os

old_folder = r'D:\Pu\20211230-P_brain_M1_nonclear\Alignment'

before_position_file = os.path.join(old_folder, '10x_positions_before.txt')

new_folder = r'D:\Pu\20220109-CTP11-1000_CTP12-DNA_from_1230\Alignment'

after_position_file = os.path.join(new_folder, '10x_positions_after.txt')

In [2]:
os.path.isfile(before_position_file), os.path.isfile(after_position_file)

(True, True)

In [3]:
import numpy as np
import os, sys
# 1. alignment for manually picked points
def align_manual_points(pos_file_before, pos_file_after,
                        save=True, save_folder=None, save_filename='', verbose=True):
    """Function to align two manually picked position files, 
    they should follow exactly the same order and of same length.
    Inputs:
        pos_file_before: full filename for positions file before translation
        pos_file_after: full filename for positions file after translation
        save: whether save rotation and translation info, bool (default: True)
        save_folder: where to save rotation and translation info, None or string (default: same folder as pos_file_before)
        save_filename: filename specified to save rotation and translation points
        verbose: say something! bool (default: True)
    Outputs:
        R: rotation for positions, 2x2 array
        T: traslation of positions, array of 2
    Here's example for how to translate points
        translated_ps_before = np.dot(ps_before, R) + t
    """
    # load position_before
    if os.path.isfile(pos_file_before):
        ps_before = np.loadtxt(pos_file_before, delimiter=',')

    # load position_after
    if os.path.isfile(pos_file_after):
        ps_after = np.loadtxt(pos_file_after, delimiter=',')

    # do SVD decomposition to get best fit for rigid-translation
    c_before = np.mean(ps_before, axis=0)
    c_after = np.mean(ps_after, axis=0)
    H = np.dot((ps_before - c_before).T, (ps_after - c_after))
    U, _, V = np.linalg.svd(H)  # do SVD
    # calcluate rotation
    R = np.dot(V, U.T).T
    if np.linalg.det(R) < 0:
        R[:, -1] = -1 * R[:, -1]
    # calculate translation
    t = - np.dot(c_before, R) + c_after
    # here's example for how to translate points
    # translated_ps_before = np.dot(ps_before, R) + t
    # save
    if save:
        if save_folder is None:
            save_folder = os.path.dirname(pos_file_before)
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        if len(save_filename) > 0:
            save_filename += '_'
        rotation_name = os.path.join(save_folder, save_filename+'rotation')
        translation_name = os.path.join(
            save_folder, save_filename+'translation')
        np.save(rotation_name, R)
        np.save(translation_name, t)

    return R, t

In [4]:
R, T = align_manual_points(before_position_file, after_position_file, save=False, save_folder=new_folder)


In [5]:
R, T

(array([[ 0.99999663, -0.0025952 ],
        [ 0.0025952 ,  0.99999663]]),
 array([-663.41808681,  539.80114533]))

# transpose 60x positions

In [6]:
old_positions = np.loadtxt(os.path.join(os.path.dirname(old_folder), 'positions_all.txt'), delimiter=',')

In [7]:
new_positions = np.dot(old_positions, R) + T
print(new_positions)

[[-4061.19405481  1439.72208144]
 [-4061.71309466  1239.72275495]
 [-4062.23213451  1039.72342846]
 [-4062.75117435   839.72410196]
 [-3862.75184786   839.20506212]
 [-3862.23280801  1039.20438861]
 [-3861.71376817  1239.2037151 ]
 [-3861.19472832  1439.2030416 ]
 [-3661.19540183  1438.68400175]
 [-3661.71444167  1238.68467526]
 [-3662.23348152  1038.68534876]
 [-3662.75252137   838.68602227]
 [-3663.27156121   638.68669578]
 [-3463.27223472   638.16765593]
 [-3462.75319487   838.16698242]
 [-3462.23415503  1038.16630892]
 [-3461.71511518  1238.16563541]
 [-3461.19607533  1438.1649619 ]
 [-3261.19674884  1437.64592205]
 [-3261.71578869  1237.64659556]
 [-3262.23482853  1037.64726907]
 [-3262.75386838   837.64794258]
 [-3263.27290823   637.64861608]
 [-3063.27358174   637.12957624]
 [-3062.75454189   837.12890273]
 [-3062.23550204  1037.12822922]
 [-3061.71646219  1237.12755571]
 [-3061.19742235  1437.12688221]
 [-2861.19809585  1436.60784236]
 [-2861.7171357   1236.60851587]
 [-2862.23

In [8]:
save_filename = os.path.join(new_folder, 'translated_positions_all.txt')
print(save_filename)
np.savetxt(save_filename, new_positions, fmt='%.2f', delimiter=',')

D:\Pu\20220109-CTP11-1000_CTP12-DNA_from_1230\Alignment\translated_positions_all.txt


# further adjust manually

In [11]:
manual_positions_before = np.array([[-3491.85,218.15],
                                    [-3449.95,3307.90],
                                    [3225.20,1039.20],
                                    [4531.30,2452.00],
                                   ])

translated_manual_positions = np.dot(manual_positions_before, R) + T
print(translated_manual_positions)

[[-4154.69018517   767.01245716]
 [-4104.77180943  3856.64331347]
 [ 2564.46798326  1570.62760921]
 [ 3874.23008241  2980.03326184]]


In [34]:
manual_positions_after = np.array([[-4188, 785],
                        [-4137,3874.5],
                        [2533,1587],
                        ])


manual_positions_after - translated_manual_positions[:len(manual_positions_after)]

array([[-33.30981483,  17.98754284],
       [-32.22819057,  17.85668653],
       [-31.46798326,  16.37239079]])

In [35]:
manual_shift = np.mean(manual_positions_after - translated_manual_positions[:len(manual_positions_after)], axis=0)
manual_shift

array([-32.33532956,  17.40554006])

In [29]:
adjusted_new_positions = new_positions + manual_shift

In [30]:
adj_save_filename = os.path.join(new_folder, 'adjusted_translated_positions_all.txt')
print(adj_save_filename)
np.savetxt(adj_save_filename, adjusted_new_positions, fmt='%.2f', delimiter=',')

D:\Pu\20220109-CTP11-1000_CTP12-DNA_from_1230\Alignment\adjusted_translated_positions_all.txt
